####DERIVANDO DADOS NO PROJETO KHIPO

####Importando tabelas

In [ ]:
import pandas as pd

In [ ]:
dfProd=pd.read_csv("product.csv",sep=";")
dfProd.head(1)

In [ ]:
dfRide=pd.read_csv("ride_v2.csv",sep=";")
dfRide.head(1)

,RideID,UserID,Schedule,Create,RideStatusID,CompanyID,ProviderID,RideProviderID,price,Updated,CategoryID,TotalUsers,Car,RideDriverLocationID,ScheduledRide
0,1685755,e15b8cc3-5a67-4630-b89f-ee69f302b582,2025-02-10 14:31:10.8858446,2025-02-10 14:31:10.9084221,1,2,NaN,NaN,0.0,2025-02-10 14:31:10.9084233,NaN,1,NaN,NaN,0


In [ ]:
dfRideAdd=pd.read_csv("rideaddress_v1.csv",sep=";",low_memory=False)
dfRideAdd.head(1)

,RideAddressID,Address,Street,Number,Neighborhood,City,State,Lat,Lng,RideAddressTypeID,RideID
0,2334277,"Rua João Pinheiro, 585 - Rua João Pinheiro - B...",Rua João Pinheiro,585,Rua João Pinheiro,NaN,Brasil,-26.329754299999998,-48.840427999999996,1,1183200


In [ ]:
dfRideEst=pd.read_csv("rideestimative_v3.csv",sep=";",low_memory=False)
dfRideEst.head(1)

,RideEstimativeID,RideID,ProductID,WaitingTime,Price,FareID,Selected,RideReasonSelectedEstimativeID,Fee
0,8619946,1183200,Flash,8,89.0,c6aaac64-5f89-4fc4-8b66-0251ec1c78a8,0,NaN,0.0


Tratando dados Nulos o visualizando tamanho das bases

In [ ]:
print(len(dfProd))
print(len(dfRide))
print(len(dfRideAdd))
print(len(dfRideEst))
print(len(dfProd.dropna()))
print(len(dfRide.dropna()))
print(len(dfRideAdd.dropna()))
print(len(dfRideEst.dropna()))


237
500000
1000000
2000000
237
14781
121971
132236


Criando coluna com dados da semana

In [ ]:
import datetime
data=pd.to_datetime(dfRide.iloc[0]["Schedule"])
print(data)
print(data.day_name())
print(data.weekday())

2025-02-10 14:31:10.885844600
Monday
0


Criando a coluna dia para receber os dados (dia da semana{segunda= 0})

In [ ]:
dia=[]
for i in range(len(dfRide)):
    data=pd.to_datetime(dfRide.iloc[i]["Schedule"])
    dia.append(data.weekday())

dfRide["Dia"]=dia
dfRide.head(5)

DIMINUINDO O TAMANHO DA BASE PARA COMEÇAR A EXPLORAR E DERIVAR.

In [ ]:
dfRide["RideID"].median()

1434271.5

In [ ]:
len(dfRide.query("RideID<=1300000"))

116782

In [ ]:
dfRide=dfRide.query("RideID<=1300000")
dfRideAdd=dfRideAdd.query("RideID<=1300000")
dfRideEst=dfRideEst.query("RideID<=1300000")
print(len(dfRide))
print(len(dfRideAdd))
print(len(dfRideEst))

116782
233564
905383


RENOMIANDO COLUNAS DATAFRAME dfRideAdd TEM A COLUNA RideAddressTypeID QUE INDICA SE O ENDEREÇO É ORIGEM(1) OU DESTINO(2) PODEMOS RENOMEAR PARA OrigDest

In [ ]:
dfRideAdd = dfRideAdd.rename(columns={'RideAddressTypeID': 'OrigDest'})
dfRideAdd.head(1)

,RideAddressID,Address,Street,Number,Neighborhood,City,State,Lat,Lng,OrigDest,RideID
0,2334277,"Rua João Pinheiro, 585 - Rua João Pinheiro - B...",Rua João Pinheiro,585,Rua João Pinheiro,NaN,Brasil,-26.329754299999998,-48.840427999999996,1,1183200


SELECIONANADO AS COLUNAS QUE QUEREMOS TRABALHAR
(dfRideAdd podemos querer trabalhar com RideID,OrigDest,Lat e Lng)

In [ ]:
dfRideAdd=dfRideAdd[["RideID","OrigDest","Lat","Lng"]]
dfRideAdd.head(1)

,RideID,OrigDest,Lat,Lng
0,1183200,1,-26.329754299999998,-48.840427999999996


Colocando tudo em uma linha Latitude e Longitude Origem e Destino e a distancia
Para isso vamos CRIAR dfRideAdd2 (Destino) e tirar o destino de dfRideAdd

In [ ]:
dfRideAdd2=dfRideAdd.query("OrigDest==2")
dfRideAdd2.head(3)

,RideID,OrigDest,Lat,Lng
1,1183200,2,-26.2554657,-48.6434197
3,1183201,2,-27.4371486,-48.39824309999999
5,1183202,2,-19.936899,-43.9401603


In [ ]:
dfRideAdd=dfRideAdd.query("OrigDest!=2")
dfRideAdd.head(3)

,RideID,OrigDest,Lat,Lng
0,1183200,1,-26.329754299999998,-48.840427999999996
2,1183201,1,-27.4919788,-48.528287999999996
4,1183202,1,-19.8495799,-44.019915999999995


Renomeando nomevamente as colunas para fazer o merge

In [ ]:
dfRideAdd=dfRideAdd.rename(columns={"Lat":"Lat1","Lng":"Lng1"})
dfRideAdd2=dfRideAdd2.rename(columns={"Lat":"Lat2","Lng":"Lng2"})

Agora de fato fazer o join

In [ ]:
dfJoinAdd = pd.merge(dfRideAdd,dfRideAdd2, left_on='RideID', right_on='RideID', how='inner')
dfJoinAdd.head(5)

,RideID,OrigDest_x,Lat1,Lng1,OrigDest_y,Lat2,Lng2
0,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197
1,1183201,1,-27.4919788,-48.528287999999996,2,-27.4371486,-48.39824309999999
2,1183202,1,-19.8495799,-44.019915999999995,2,-19.936899,-43.9401603
3,1183203,1,-23.9624233,-46.25465759999999,2,-23.8373074,-46.1321725
4,1183204,1,-10.9198019,-37.077441799999995,2,-10.9071288,-37.0877194


Agora logica de distância

In [ ]:
!pip install geopy

In [ ]:
from geopy.distance import geodesic

# Pontos: (latitude, longitude)
ponto1 = (-27.4919788, -48.528287999999996)
ponto2 = (-27.4371486, -48.39824309999999)

# Distância em km
distancia_km = geodesic(ponto1, ponto2).kilometers
print(f"Distância: {distancia_km:.2f} km")

Distância: 14.22 km


Agora adicionando a distância em toda a tabela

In [ ]:
Dist=[]

for i in range(len(dfJoinAdd)):
    lat1=float(dfJoinAdd.iloc[i]["Lat1"].replace(",","."))
    lng1=float(dfJoinAdd.iloc[i]["Lng1"].replace(",","."))
    lat2=float(dfJoinAdd.iloc[i]["Lat2"].replace(",","."))
    lng2=float(dfJoinAdd.iloc[i]["Lng2"].replace(",","."))
    ponto1 = (lat1, lng1)
    ponto2 = (lat2, lng2)
    distkm = geodesic(ponto1, ponto2).kilometers
    Dist.append(distkm)

dfJoinAdd["Distancia"]=Dist

dfJoinAdd.head(5)

,RideID,OrigDest_x,Lat1,Lng1,OrigDest_y,Lat2,Lng2,Distancia
0,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197,21.327087
1,1183201,1,-27.4919788,-48.528287999999996,2,-27.4371486,-48.39824309999999,14.217724
2,1183202,1,-19.8495799,-44.019915999999995,2,-19.936899,-43.9401603,12.774728
3,1183203,1,-23.9624233,-46.25465759999999,2,-23.8373074,-46.1321725,18.643943
4,1183204,1,-10.9198019,-37.077441799999995,2,-10.9071288,-37.0877194,1.796511


Separando o dia da semana vinculando pelo RideID
Já usando o melhor modo de treinar a ML.

In [ ]:
dfRide3 = dfRide[["RideID","Dia"]]
print(dfRide3)

         RideID  Dia
383218  1300000    3
383219  1299999    3
383220  1299998    3
383221  1299997    3
383222  1299996    3
...         ...  ...
499995  1183204    1
499996  1183203    1
499997  1183202    1
499998  1183201    1
499999  1183200    1

[116782 rows x 2 columns]


In [ ]:
#dfJoinAdd = pd.merge(dfRideAdd,dfRideAdd2, left_on='RideID', right_on='RideID', how='inner')
#dfJoinAdd.head(5)

dfJoinAdd2 = pd.merge(dfJoinAdd, dfRide3, left_on = 'RideID', right_on ='RideID', how='inner')
dfJoinAdd2.head(5)

,RideID,OrigDest_x,Lat1,Lng1,OrigDest_y,Lat2,Lng2,Distancia,Dia
0,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197,21.327087,1
1,1183201,1,-27.4919788,-48.528287999999996,2,-27.4371486,-48.39824309999999,14.217724,1
2,1183202,1,-19.8495799,-44.019915999999995,2,-19.936899,-43.9401603,12.774728,1
3,1183203,1,-23.9624233,-46.25465759999999,2,-23.8373074,-46.1321725,18.643943,1
4,1183204,1,-10.9198019,-37.077441799999995,2,-10.9071288,-37.0877194,1.796511,1


Selecionando os produtos para treinar a ML e imprimindo a quantidade de corridas para treinamento.

In [ ]:
dfprodutos = dfRideEst.query("ProductID in ['UberX', '99POP', 'Flash', 'Confort', 'pop99','poupa99', 'Bag', 'UberFlash', '', '', '7da40add1baab37e19cae629f6907b13']")
#7da40add1baab37e19cae629f6907b13 (Cabify Lite Corp)
#dfprodutos.head()
print(len(dfprodutos))

421534


Separando o preço por RideID

In [ ]:
dftprodutos = dfprodutos[["RideID","Price"]]
print(dftprodutos)

         RideID   Price
0       1183200   89.00
1       1183200   89.00
3       1183200  170.21
4       1183200  170.21
7       1183201   31.50
...         ...     ...
905373  1299999   51.50
905376  1300000   20.33
905377  1300000   23.62
905380  1300000   20.00
905382  1300000   20.00

[421534 rows x 2 columns]


In [ ]:
dfJoinAdd3 = pd.merge(dfJoinAdd2, dftprodutos, left_on = 'RideID', right_on ='RideID', how='inner')
#dfJoinAdd3.query("Dia==6")
dfJoinAdd3.head(5)

,RideID,OrigDest_x,Lat1,Lng1,OrigDest_y,Lat2,Lng2,Distancia,Dia,Price
0,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197,21.327087,1,89.00
1,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197,21.327087,1,89.00
2,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197,21.327087,1,170.21
3,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197,21.327087,1,170.21
4,1183201,1,-27.4919788,-48.528287999999996,2,-27.4371486,-48.39824309999999,14.217724,1,31.50


In [ ]:
dfRide['schedule']= pd.to_datetime(dfRide['Schedule'])
dfRide['schedule_time'] = dfRide['schedule'].dt.time
dfRide.head(1)

,RideID,UserID,Schedule,Create,RideStatusID,CompanyID,ProviderID,RideProviderID,price,Updated,CategoryID,TotalUsers,Car,RideDriverLocationID,ScheduledRide,Dia,schedule,schedule_time
383218,1300000,acbb3e9f-62a8-4f01-b04f-f58058728fc4,2021-09-30 16:01:11.5153413,2021-09-30 16:01:11.5488417,2,40,NaN,NaN,20.0,2021-09-30 16:01:23.4585643,NaN,1,NaN,NaN,0,3,2021-09-30 16:01:11.515341300,16:01:11.515341


Separando a hora da coluna schedule da dfRideId

In [ ]:
dfRideT=dfRide[["RideID","schedule_time"]]
dfRideT.head(1)

,RideID,schedule_time
383218,1300000,16:01:11.515341


Fazendo merge da schedulo(hora de inicio)

In [ ]:
dfJoinAdd3 = pd.merge(dfJoinAdd3, dfRideT, left_on = 'RideID', right_on ='RideID', how='inner')
dfJoinAdd3.head(1)

,RideID,OrigDest_x,Lat1,Lng1,OrigDest_y,Lat2,Lng2,Distancia,Dia,Price,schedule_time
0,1183200,1,-26.329754299999998,-48.840427999999996,2,-26.2554657,-48.6434197,21.327087,1,89.0,10:09:45.628170


#### Extrair a base pronta para ML

In [ ]:
dfJoinAdd3.to_csv('dfJoinAdd3.csv', index=False)

#### Treinamento da Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X = dfJoinAdd3[["OrigDest_x","Lat1","Lng1","OrigDest_y","Lat2","Lng2","Distancia","Dia","schedule_time"]]
y = dfJoinAdd3["Price"]

SEED = 20

treino_x, teste_x, treino_y, teste_y = train_test_split(X, y, test_size=0.3, random_state=SEED) # Linha adicionada
#coluna limpa:

# Convertendo colunas de latitude e longitude para o formato correto
for col in ["Lat1", "Lng1", "Lat2", "Lng2"]:
    treino_x[col] = treino_x[col].astype(str).str.replace(',', '.').astype(float)
    teste_x[col] = teste_x[col].astype(str).str.replace(',', '.').astype(float)

# Convertendo 'schedule_time' para segundos desde a meia-noite
treino_x['schedule_time'] = treino_x['schedule_time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
teste_x['schedule_time'] = teste_x['schedule_time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

# Agora você pode treinar o modelo:
model = RandomForestRegressor(n_estimators=100, random_state=SEED)
model.fit(treino_x , treino_y)

predicao_y = model.predict(teste_x)
r2 = r2_score(teste_y, predicao_y)
mse = mean_squared_error(teste_y, predicao_y)
mae = mean_absolute_error(teste_y, predicao_y)

print(f"Erro médio absoluto: {mae}")
print(f"Erro quadrático médio: {mse}")

Erro médio absoluto: 6.511030321116462
Erro quadrático médio: 364.19364178642485


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# 2 - Definindo o modelo:
modelo = GradientBoostingRegressor()
modelo.fit(treino_x, treino_y)

GradientBoostingRegressor()

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# 2 - Aplicação do Teste:
#predicao_y = modelo.predict(teste_x)
predicao_y = modelo.predict(teste_x)

# 3 - Avaliação do Teste (com todos os modelos de Metrics):
r2 = r2_score(teste_y, predicao_y)

mse = mean_squared_error(teste_y, predicao_y)
mae = mean_absolute_error(teste_y, predicao_y)

# 4 - Mostrar os resultados:

print(f"R2 (Acuracia): {r2*100:.2f}%")
print(f"MSE : {mse:.2f}")
print(f"MAE : {mae:.2f}")

R2 (Acuracia): 90.12%
MSE : 435.93
MAE : 7.78


####Logica Final pedindo endereço

In [ ]:
!pip install --upgrade WazeRouteCalculator

In [ ]:
from WazeRouteCalculator import WazeRouteCalculator, WRCError
from geopy.geocoders import Nominatim
from datetime import datetime
import pandas as pd
import warnings
import unicodedata

warnings.filterwarnings("ignore")

# Inicializa geolocalizador
geolocator = Nominatim(user_agent="predictor-app")

def limpar_endereco(endereco):
    nfkd = unicodedata.normalize('NFKD', endereco)
    return u"".join([c for c in nfkd if not unicodedata.combining(c)])

def calcular_distancia_waze(origem, destino, region='EU'):
    try:
        lat1, lng1 = endereco_para_coordenadas(origem)
        lat2, lng2 = endereco_para_coordenadas(destino)

        origem_coord = f"{lat1},{lng1}"
        destino_coord = f"{lat2},{lng2}"

        rota = WazeRouteCalculator(origem_coord, destino_coord, region)
        tempo_min, distancia_km = rota.calc_route_info(real_time=False)  # ou real_time=True, se quiser tráfego atual

        return distancia_km, tempo_min
    except WRCError as e:
        raise ValueError(f"Erro ao calcular rota Waze: {e}")


def endereco_para_coordenadas(endereco):
    endereco_limpo = limpar_endereco(endereco)
    location = geolocator.geocode(endereco_limpo, timeout=15)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"Não foi possível geocodificar o endereço: {endereco}")

def gerar_features(end_origem, end_destino, horario=None):
    lat1, lng1 = endereco_para_coordenadas(end_origem)
    lat2, lng2 = endereco_para_coordenadas(end_destino)
    distancia, tempo = calcular_distancia_waze(end_origem, end_destino)

    now = horario or datetime.now()
    schedule_time = now.hour * 3600 + now.minute * 60 + now.second
    dia = now.weekday()

    orig_dest_x = abs(hash(end_origem)) % (10 ** 8)
    orig_dest_y = abs(hash(end_destino)) % (10 ** 8)

    df = pd.DataFrame([{
        "OrigDest_x": orig_dest_x,
        "Lat1": lat1,
        "Lng1": lng1,
        "OrigDest_y": orig_dest_y,
        "Lat2": lat2,
        "Lng2": lng2,
        "Distancia": distancia,
        "Tempo": tempo,
        "Dia": dia,
        "schedule_time": schedule_time
    }])

    return df

def prever_preco(end_origem, end_destino):
    dados = gerar_features(end_origem, end_destino)
    dados_para_prever = dados.drop(columns=["Tempo"])
    preco_estimado = model.predict(dados_para_prever)[0]

    dados["Preco_Previsto"] = preco_estimado
    dados["Endereco_Origem"] = end_origem
    dados["Endereco_Destino"] = end_destino
    return dados

# ================================

if __name__ == "__main__":
    print("=== Previsão de Preço Uber ===")
    origem = input("Digite o endereço de ORIGEM: ").strip()
    destino = input("Digite o endereço de DESTINO: ").strip()

    try:
        resultado = prever_preco(origem, destino)
        preco = resultado["Preco_Previsto"].values[0]
        distancia = resultado["Distancia"].values[0]
        tempo = resultado["Tempo"].values[0]

        print(f"\n🚗 Preço estimado da corrida: R$ {preco:.2f}")
        print(f"📍 Distância estimada: {distancia:.2f} km")
        print(f"⏱️ Tempo estimado: {tempo:.0f} minutos")

    except Exception as e:
        print(f"\n❌ Erro ao calcular o preço: {e}")


=== Previsão de Preço Uber ===
Digite o endereço de ORIGEM: Rua Itajai 125 - são paulo
Digite o endereço de DESTINO: rua general joão sarmento pimentel 86 - são paulo

🚗 Preço estimado da corrida: R$ 81.57
📍 Distância estimada: 26.70 km
⏱️ Tempo estimado: 41 minutos
